In [93]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn import linear_model
import matplotlib.pyplot as plt
import nltk as nltk
%matplotlib inline

#Import CSV files that contains each messages and rating from differnt groups of participants as a Pandas Dataframe
codings=pd.read_csv('data/batch_all.csv', index_col=0, parse_dates=True)
codings.head()

,Id,Request#,EN_1,EN_2,EN_3,EN_4,EN_5,EN_6,EN_7,EN_8,...,CH1800,CH1900,CH2000,CH2100,CH2200,CH2300,CH2400,CH2500,CH2600,CH2700
Num,,,,,,,,,,,,,,,,,,,,,
1,448758,Answered at <url>. Maybe you are not familiar ...,-10.0,NaN,NaN,NaN,NaN,-3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,367359,"Btw, ignoring the London stations it looks lik...",-7.0,NaN,NaN,NaN,NaN,-4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,622653,Because Wikipedia is an encylopaedia not a fre...,-12.0,NaN,NaN,NaN,NaN,-2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,350825,'Images with iconic status or historical impor...,-12.0,NaN,NaN,NaN,NaN,-8.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,348828,May I also ask how CheckUser became involved i...,-11.0,NaN,NaN,NaN,NaN,-6.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
headers=list(codings.columns.values)
# print(headers)

In [95]:
#Split up the files by participants' language groups by the headers
en_headers=[]
ch_headers=[]

for h in headers:
    if h.startswith('EN'):
        en_headers.append(h)
    elif h.startswith('CH'):
        ch_headers.append(h)
print(en_headers, ch_headers)
print(len(en_headers)+len(ch_headers))

['EN_1', 'EN_2', 'EN_3', 'EN_4', 'EN_5', 'EN_6', 'EN_7', 'EN_8', 'EN_9', 'EN_10', 'EN_11', 'EN_12', 'EN_13', 'EN_14', 'EN_15', 'EN_16', 'EN_17', 'EN_18', 'EN_19', 'EN_20', 'EN_21', 'EN_22', 'EN_23', 'EN_24', 'EN_25', 'EN_26', 'EN_27', 'EN_28', 'EN_29', 'EN_30', 'EN_31', 'EN_10.1', 'EN_20.1', 'EN_30.1', 'EN_40', 'EN_50', 'EN_60', 'EN_70', 'EN_80', 'EN_90', 'EN_100', 'EN_110', 'EN_120', 'EN_130', 'EN_140', 'EN_150', 'EN_160', 'EN_170', 'EN_180', 'EN_190', 'EN_200', 'EN_210', 'EN_220', 'EN_230', 'EN_240', 'EN_250', 'EN_260', 'EN_270', 'EN_280', 'EN_290', 'EN_300', 'EN_310', 'EN_320', 'EN_330', 'EN_340', 'EN_350', 'EN100', 'EN200', 'EN300', 'EN400', 'EN500', 'EN600', 'EN700', 'EN800', 'EN900', 'EN1000', 'EN1100', 'EN1200', 'EN1300', 'EN1400', 'EN1500', 'EN1600', 'EN1700', 'EN1800', 'EN1900', 'EN2000', 'EN2100', 'EN2200', 'EN2300', 'EN2400'] ['CH_1', 'CH_2', 'CH_3', 'CH_4', 'CH_5', 'CH_6', 'CH_7', 'CH_8', 'CH_9', 'CH_10', 'CH_11', 'CH_12', 'CH_13', 'CH_14', 'CH_15', 'CH_16', 'CH_17', 'CH_18

In [96]:
#calculating z scores by individuals 

en_headers_zscore=[]
ch_headers_zscore=[]

for col in en_headers:
    col_zscore=col+'_zscore'
    codings[col_zscore]=(codings[col]-codings[col].mean())/codings[col].std(ddof=0)
    en_headers_zscore.append(col_zscore)
    

for col in ch_headers:
    col_zscore=col+'_zscore'
    codings[col_zscore]=(codings[col]-codings[col].mean())/codings[col].std(ddof=0)
    ch_headers_zscore.append(col_zscore)

codings.columns.values

array(['Id', 'Request#', 'EN_1', 'EN_2', 'EN_3', 'EN_4', 'EN_5', 'EN_6',
       'EN_7', 'EN_8', 'EN_9', 'EN_10', 'EN_11', 'EN_12', 'EN_13',
       'EN_14', 'EN_15', 'EN_16', 'EN_17', 'EN_18', 'EN_19', 'EN_20',
       'EN_21', 'EN_22', 'EN_23', 'EN_24', 'EN_25', 'EN_26', 'EN_27',
       'EN_28', 'EN_29', 'EN_30', 'EN_31', 'CH_1', 'CH_2', 'CH_3', 'CH_4',
       'CH_5', 'CH_6', 'CH_7', 'CH_8', 'CH_9', 'CH_10', 'CH_11', 'CH_12',
       'CH_13', 'CH_14', 'CH_15', 'CH_16', 'CH_17', 'CH_18', 'CH_19',
       'CH_20', 'CH_21', 'CH_22', 'CH_23', 'EN_10.1', 'EN_20.1',
       'EN_30.1', 'EN_40', 'EN_50', 'EN_60', 'EN_70', 'EN_80', 'EN_90',
       'EN_100', 'EN_110', 'EN_120', 'EN_130', 'EN_140', 'EN_150',
       'EN_160', 'EN_170', 'EN_180', 'EN_190', 'EN_200', 'EN_210',
       'EN_220', 'EN_230', 'EN_240', 'EN_250', 'EN_260', 'EN_270',
       'EN_280', 'EN_290', 'EN_300', 'EN_310', 'EN_320', 'EN_330',
       'EN_340', 'EN_350', 'CH_10.1', 'CH_20.1', 'CH_30', 'CH_40',
       'CH_50', 'CH_60', 'CH_

In [97]:
#make binary labels 
en_headers_binary=[]
ch_headers_binary=[]

for col in en_headers:
    col2=codings[col]
    col_list=[]
    for i in col2:
        if i>0:
            col_list.append(1)
        elif i<0:
            col_list.append(-1)
        else:
            col_list.append(i)
    
    col_binary=col+'_binary'
    codings[col_binary]=col_list

In [98]:
for col in ch_headers:
    col2=codings[col]
    col_list=[]
    for i in col2:
        if i>0:
            col_list.append(1)
        elif i<0:
            col_list.append(-1)
        else:
            col_list.append(i)
    
    col_binary=col+'_binary'
    codings[col_binary]=col_list

In [99]:
len(codings.columns.values)

521

In [102]:
codings.to_csv('all_data.csv')